In [1]:
import pandas as pd
import tensorflow as tf
import lib.dataProcess as dp
import lib.deepLearning as dl

In [35]:
file_names = ["OV_Var_200.csv", "OV_Diff_200.csv", "OV_CV_200.csv", "OV_Annotation3000_200.csv", "OV_Annotation40.csv"]
file_name = file_names[3]

In [36]:
raw_data = pd.read_csv('../GEO_input_ensemble_CV_500.csv')

In [3]:
shuffled_data = raw_data
shuffled_data['index'] = dp.shuffle_index(shuffled_data)

In [4]:
fivefold = dp.n_fold(raw_data, 'index', 5)
# make index as rep of 1:5

In [5]:
xdata_five, ydata_five = dp.divide_xy_train(fivefold, 'result', True, 1, -3)
train_x, test_x = dp.train_test(xdata_five, 0)
train_y, test_y = dp.train_test(ydata_five, 0)
train_y = dp.one_hot_encoder(train_y)
test_y = dp.one_hot_encoder(test_y)
val_x, test_x = dp.test_validation(test_x)
val_y, test_y = dp.test_validation(test_y)
# devide train & test

Number of Class : 2
Number of Class : 2


C:\Git\BDLAB_ML_Library\lib\dataProcess.py:110: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  y_data_n.append(data.loc[:,key].as_matrix())
C:\Git\BDLAB_ML_Library\lib\dataProcess.py:111: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  x_data_n.append(data.iloc[:,x_start:x_end].as_matrix())


In [16]:
nodes = [200,100, 50]
learning_rate = 0.001
batch_size = 100

X, Y, layers, logits, phase, hypothesis, cost, train, predicted, correct_prediction, accuracy, keep_prob = dl.set_model_dropout(train_x, train_y, nodes , learning_rate)

#set hyperparameters - node, learning rate, batch size
#what is keep_prob?
#make place holders. These are not real variables, just spaces for variable.

<tf.Tensor 'Placeholder_1:0' shape=(?, 500) dtype=float32>

In [18]:

best_train_acc = 0
best_val_acc = 0
best_test_acc = 0 
best_cost = float("inf")
save_path_dir ='./'
count = 0
model_num = 0

saver = tf.train.Saver()
with tf.Session() as sess:
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())
    stop_switch = True
    step = 0
    
    #if condition 
    while stop_switch:
        total_num = int(len(train_x) / batch_size)
        for k in range(total_num):
            #cut data as large as batch_size.
            batch_x = train_x[k * batch_size:(k + 1) * batch_size]
            batch_y = train_y[k * batch_size:(k + 1) * batch_size]
            sess.run(train, feed_dict={X: batch_x, Y: batch_y, keep_prob: 0.5 , phase:True})

        #feed_dict - place holder is just 'space', feed_dict means supply real data into place holder.
        train_h, train_c, train_p, train_a = sess.run( [hypothesis, cost, predicted, accuracy], feed_dict={X: train_x, Y: train_y, keep_prob: 1 , phase:False})
        val_h, val_c, val_p, val_a = sess.run([hypothesis, cost, predicted, accuracy], feed_dict={X: val_x, Y: val_y, keep_prob: 1 , phase:False})
        if step % 20 == 0 :
            print("train acc : ", train_a, "validation acc : ", val_a, "train_cost", train_c)
        step += 1
        
        #condition 1: new best condition. 
        #calculated cost(val_c) is smaller than before's(best_cost), save present condition and initialize count.
        if best_cost > val_c :
            best_train_acc = train_a
            best_val_acc = val_a
            best_cost = val_c
            count = 0
            save_path = saver.save(sess, save_path_dir + 'model'+str(model_num)+'.ckpt')

        #condition 2: cannot find best condition
        #when cost is get worse and worse(10 times), finish learning.
        elif count > 10 :
            stop_switch = False
            print("Learning Finished!! \n")
        
        #condition 3: condition is not best, but not yet finish.
        else:
            count += 1

    print("Training Accuracy : ", best_train_acc,  "Validation Accuracy : ", best_val_acc)

    saver.restore(sess, save_path)

    test_h, test_p, test_a = sess.run([hypothesis, predicted, accuracy],
                                      feed_dict={X: test_x, Y: test_y, keep_prob:1.0 , phase:False})
    print("\nTest Accuracy: ", test_a)
    best_test_acc = test_a


    model_num += 1                    
    

train acc :  0.8695652 validation acc :  0.88039213 train_cost 0.35921618
Learning Finished!! 

Training Accuracy :  0.96697694 Validation Accuracy :  0.95392156
INFO:tensorflow:Restoring parameters from ./model0.ckpt

Test Accuracy:  0.9588235


In [19]:
data_x, data_y = dp.divide_xy_test(raw_data, 'result', 1,-3)
ids = raw_data['patient']

C:\Git\BDLAB_ML_Library\lib\dataProcess.py:117: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  ydata = data.loc[:, key].as_matrix()
C:\Git\BDLAB_ML_Library\lib\dataProcess.py:118: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  xdata = data.iloc[:, x_start:x_end].as_matrix()


In [20]:
saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess, './model0.ckpt')
    test_h, test_p = sess.run([hypothesis, predicted], feed_dict={X: data_x, Y: dp.one_hot_encoder(data_y), keep_prob:1.0 , phase:False})

INFO:tensorflow:Restoring parameters from ./model0.ckpt
Number of Class : 2


In [21]:
dp.prediction_probability(test_h, test_p, data_y, ids)

In [12]:
len(test_h)

4408

In [22]:
len(data_y)

9883

In [23]:
len(ids)

9883

In [24]:
len(test_p)

9883

In [25]:
test_p

array([1, 0, 0, ..., 0, 0, 0], dtype=int64)

In [26]:
test_h

array([[3.2725313e-03, 9.9672747e-01],
       [7.2946763e-01, 2.7053237e-01],
       [9.9924636e-01, 7.5364835e-04],
       ...,
       [9.9945074e-01, 5.4927211e-04],
       [9.9957663e-01, 4.2333794e-04],
       [9.9833894e-01, 1.6610039e-03]], dtype=float32)

array([1, 0, 0, ..., 0, 0, 0], dtype=int64)